In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

In [0]:
# Initialize Spark Session
spark = SparkSession.builder \
    .appName("CustomerPurchaseEDA") \
    .config("spark.sql.adaptive.enabled", "true") \
    .config("spark.sql.adaptive.coalescePartitions.enabled", "true") \
    .getOrCreate()

In [0]:
# Load the dataset
df = spark.read.option("header", "true").option("inferSchema", "true").csv("/Volumes/workspace/default/project/customer_purchase_dataset.csv")

In [0]:
print("=== BASIC DATA INFORMATION ===")
print(f"Dataset shape: {df.count()} rows, {len(df.columns)} columns")
df.printSchema()

=== BASIC DATA INFORMATION ===
Dataset shape: 1000 rows, 8 columns
root
 |-- customer_id: integer (nullable = true)
 |-- order_date: date (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- product_category: string (nullable = true)
 |-- payment_type: string (nullable = true)
 |-- delivery_location: string (nullable = true)
 |-- days_since_last_purchase: integer (nullable = true)
 |-- repeat_purchase_next_30_days: integer (nullable = true)



In [0]:
print("\n=== DATA PREVIEW ===")
df.show(10, truncate=False)


=== DATA PREVIEW ===
+-----------+----------+------------+----------------+--------------+-----------------+------------------------+----------------------------+
|customer_id|order_date|total_amount|product_category|payment_type  |delivery_location|days_since_last_purchase|repeat_purchase_next_30_days|
+-----------+----------+------------+----------------+--------------+-----------------+------------------------+----------------------------+
|58         |2023-01-26|37206.75    |Groceries       |Credit Card   |Lalitpur         |26                      |0                           |
|217        |2023-02-02|1974.96     |Groceries       |Credit Card   |Biratnagar       |154                     |0                           |
|288        |2023-07-23|35942.97    |Home Appliances |Digital Wallet|Lalitpur         |114                     |1                           |
|4          |2023-06-13|35057.9     |Clothing        |Bank Transfer |Lalitpur         |55                      |1             

In [0]:
print("\n=== MISSING VALUES ANALYSIS ===")
missing_counts = df.select([
    count(when(col(c).isNull(), c)).alias(c) for c in df.columns
])
missing_counts.show()


=== MISSING VALUES ANALYSIS ===
+-----------+----------+------------+----------------+------------+-----------------+------------------------+----------------------------+
|customer_id|order_date|total_amount|product_category|payment_type|delivery_location|days_since_last_purchase|repeat_purchase_next_30_days|
+-----------+----------+------------+----------------+------------+-----------------+------------------------+----------------------------+
|          0|         0|           0|               0|           0|                0|                       0|                           0|
+-----------+----------+------------+----------------+------------+-----------------+------------------------+----------------------------+



In [0]:
print("\n=== BASIC STATISTICS ===")
df.describe().show()


=== BASIC STATISTICS ===
+-------+-----------------+------------------+----------------+--------------+-----------------+------------------------+----------------------------+
|summary|      customer_id|      total_amount|product_category|  payment_type|delivery_location|days_since_last_purchase|repeat_purchase_next_30_days|
+-------+-----------------+------------------+----------------+--------------+-----------------+------------------------+----------------------------+
|  count|             1000|              1000|            1000|          1000|             1000|                    1000|                        1000|
|   mean|          149.671| 25527.73304999998|            NULL|          NULL|             NULL|                  89.439|                       0.532|
| stddev|88.30700360608944|14172.471314222195|            NULL|          NULL|             NULL|       51.61123907637754|          0.4992246240173108|
|    min|                1|             508.4|        Clothing| Bank

In [0]:
print("\n=== TARGET VARIABLE DISTRIBUTION ===")
target_dist = df.groupBy("repeat_purchase_next_30_days").count().collect()
for row in target_dist:
    print(f"Class {row[0]}: {row[1]} samples")



class_imbalance = df.agg(
    avg("repeat_purchase_next_30_days").alias("positive_rate")
).collect()[0][0]

print(f"Positive class rate: {class_imbalance:.3f}")


=== TARGET VARIABLE DISTRIBUTION ===
Class 1: 532 samples
Class 0: 468 samples
Positive class rate: 0.532


In [0]:
print("\n=== CATEGORICAL FEATURES ANALYSIS ===")

# Product category distribution
print("Product Category Distribution:")
df.groupBy("product_category").agg(
    count("*").alias("count"),
    avg("repeat_purchase_next_30_days").alias("repeat_rate")
).orderBy(desc("count")).show()

# Payment type analysis
print("Payment Type Distribution:")
df.groupBy("payment_type").agg(
    count("*").alias("count"),
    avg("repeat_purchase_next_30_days").alias("repeat_rate")
).orderBy(desc("count")).show()

# Delivery location analysis
print("Delivery Location Distribution:")
df.groupBy("delivery_location").agg(
    count("*").alias("count"),
    avg("repeat_purchase_next_30_days").alias("repeat_rate")
).orderBy(desc("count")).show()


=== CATEGORICAL FEATURES ANALYSIS ===
Product Category Distribution:
+----------------+-----+------------------+
|product_category|count|       repeat_rate|
+----------------+-----+------------------+
|        Clothing|  223|0.5201793721973094|
| Home Appliances|  207|0.5217391304347826|
|       Groceries|  195| 0.517948717948718|
|      Stationery|  191|0.5759162303664922|
|     Electronics|  184|0.5271739130434783|
+----------------+-----+------------------+

Payment Type Distribution:
+--------------+-----+------------------+
|  payment_type|count|       repeat_rate|
+--------------+-----+------------------+
|   Credit Card|  267|0.5056179775280899|
| Bank Transfer|  248|0.5524193548387096|
|          Cash|  246|0.5528455284552846|
|Digital Wallet|  239|0.5188284518828452|
+--------------+-----+------------------+

Delivery Location Distribution:
+-----------------+-----+------------------+
|delivery_location|count|       repeat_rate|
+-----------------+-----+------------------+
| 

In [0]:
print("\n=== NUMERICAL FEATURES ANALYSIS ===")

# Total amount statistics by target
print("Total Amount by Target Class:")
df.groupBy("repeat_purchase_next_30_days").agg(
    avg("total_amount").alias("avg_amount"),
    stddev("total_amount").alias("std_amount"),
    min("total_amount").alias("min_amount"),
    max("total_amount").alias("max_amount")
).show()

# Days since last purchase analysis
print("Days Since Last Purchase by Target Class:")
df.groupBy("repeat_purchase_next_30_days").agg(
    avg("days_since_last_purchase").alias("avg_days"),
    stddev("days_since_last_purchase").alias("std_days"),
    min("days_since_last_purchase").alias("min_days"),
    max("days_since_last_purchase").alias("max_days")
).show()


=== NUMERICAL FEATURES ANALYSIS ===
Total Amount by Target Class:
+----------------------------+------------------+------------------+----------+----------+
|repeat_purchase_next_30_days|        avg_amount|        std_amount|min_amount|max_amount|
+----------------------------+------------------+------------------+----------+----------+
|                           1|25731.204060150405| 14328.61815112422|    674.88|  49922.46|
|                           0|25296.436944444424|14004.571359429498|     508.4|  49979.95|
+----------------------------+------------------+------------------+----------+----------+

Days Since Last Purchase by Target Class:
+----------------------------+-----------------+-----------------+--------+--------+
|repeat_purchase_next_30_days|         avg_days|         std_days|min_days|max_days|
+----------------------------+-----------------+-----------------+--------+--------+
|                           1|89.54323308270676|51.31080299535794|       1|     180|
|   

In [0]:
print("\n=== OUTLIER DETECTION ===")

# Calculate quartiles for numerical columns
numerical_cols = ["total_amount", "days_since_last_purchase"]

for col_name in numerical_cols:
    quartiles = df.approxQuantile(col_name, [0.25, 0.5, 0.75], 0.0)
    q1, median, q3 = quartiles
    iqr = q3 - q1
    lower_bound = q1 - 1.5 * iqr
    upper_bound = q3 + 1.5 * iqr
    
    outliers = df.filter((col(col_name) < lower_bound) | (col(col_name) > upper_bound)).count()
    print(f"{col_name}: Q1={q1:.2f}, Median={median:.2f}, Q3={q3:.2f}")
    print(f"  IQR={iqr:.2f}, Outliers: {outliers} ({outliers/df.count()*100:.1f}%)")


=== OUTLIER DETECTION ===
total_amount: Q1=13122.28, Median=25823.45, Q3=37510.09
  IQR=24387.81, Outliers: 0 (0.0%)
days_since_last_purchase: Q1=43.00, Median=89.00, Q3=132.00
  IQR=89.00, Outliers: 0 (0.0%)


In [0]:

print("\n=== TEMPORAL ANALYSIS ===")

# Convert order_date to date type and extract features
df_temporal = df.withColumn("order_date", to_date(col("order_date"), "yyyy-MM-dd")) \
               .withColumn("order_year", year("order_date")) \
               .withColumn("order_month", month("order_date")) \
               .withColumn("order_day_of_week", dayofweek("order_date"))

print("Orders by Year:")
df_temporal.groupBy("order_year").agg(
    count("*").alias("total_orders"),
    avg("repeat_purchase_next_30_days").alias("repeat_rate")
).orderBy("order_year").show()

print("Orders by Month:")
df_temporal.groupBy("order_month").agg(
    count("*").alias("total_orders"),
    avg("repeat_purchase_next_30_days").alias("repeat_rate")
).orderBy("order_month").show()

print("Orders by Day of Week (1=Sunday):")
df_temporal.groupBy("order_day_of_week").agg(
    count("*").alias("total_orders"),
    avg("repeat_purchase_next_30_days").alias("repeat_rate")
).orderBy("order_day_of_week").show()


=== TEMPORAL ANALYSIS ===
Orders by Year:
+----------+------------+------------------+
|order_year|total_orders|       repeat_rate|
+----------+------------+------------------+
|      2023|         606| 0.533003300330033|
|      2024|         394|0.5304568527918782|
+----------+------------+------------------+

Orders by Month:
+-----------+------------+-------------------+
|order_month|total_orders|        repeat_rate|
+-----------+------------+-------------------+
|          1|         122|0.47540983606557374|
|          2|         103| 0.5048543689320388|
|          3|          94| 0.5531914893617021|
|          4|         106| 0.5566037735849056|
|          5|          94| 0.5425531914893617|
|          6|         106| 0.5754716981132075|
|          7|         122| 0.5081967213114754|
|          8|          54| 0.6111111111111112|
|          9|          51| 0.5490196078431373|
|         10|          49|0.42857142857142855|
|         11|          56| 0.5178571428571429|
|         1

In [0]:
print("\n=== CORRELATION ANALYSIS ===")

# Convert to Pandas for correlation matrix (small dataset assumption)
pandas_df = df.select("total_amount", "days_since_last_purchase", "repeat_purchase_next_30_days").toPandas()

correlation_matrix = pandas_df.corr()
print("Correlation Matrix:")
print(correlation_matrix)


=== CORRELATION ANALYSIS ===
Correlation Matrix:
                              total_amount  ...  repeat_purchase_next_30_days
total_amount                      1.000000  ...                      0.015315
days_since_last_purchase         -0.013527  ...                      0.002154
repeat_purchase_next_30_days      0.015315  ...                      1.000000

[3 rows x 3 columns]


In [0]:
print("\n=== FEATURE INSIGHTS FOR ENGINEERING ===")

# Customer-level aggregations preview
customer_stats = df.groupBy("customer_id").agg(
    count("*").alias("total_orders"),
    avg("total_amount").alias("avg_order_value"),
    sum("total_amount").alias("total_spent"),
    avg("days_since_last_purchase").alias("avg_days_between"),
    first("repeat_purchase_next_30_days").alias("target")
)

print("Customer-level Statistics (sample):")
customer_stats.show(10)

print("Customers with Multiple Orders:")
multi_order_customers = customer_stats.filter(col("total_orders") > 1).count()
total_customers = customer_stats.count()
print(f"Multi-order customers: {multi_order_customers}/{total_customers} ({multi_order_customers/total_customers*100:.1f}%)")



=== FEATURE INSIGHTS FOR ENGINEERING ===
Customer-level Statistics (sample):
+-----------+------------+------------------+------------------+------------------+------+
|customer_id|total_orders|   avg_order_value|       total_spent|  avg_days_between|target|
+-----------+------------+------------------+------------------+------------------+------+
|         94|           2|         38436.015|          76872.03|              90.5|     0|
|         29|           6|30403.003333333338|182418.02000000002|51.666666666666664|     0|
|        284|           8|26746.531250000004|213972.25000000003|            64.875|     1|
|         88|           5|28865.703999999998|         144328.52|              81.6|     0|
|        131|           1|          15734.74|          15734.74|              29.0|     1|
|        148|           4|        19173.3825|          76693.53|              26.5|     1|
|        279|           6|24256.280000000002|145537.68000000002|122.83333333333333|     0|
|        195

In [0]:
print("\n=== DATA QUALITY CHECKS ===")

# Check for duplicate customer_ids (if this should be unique per row)
duplicate_customers = df.groupBy("customer_id").count().filter(col("count") > 1).count()
print(f"Customers with multiple rows: {duplicate_customers}")

# Check date range
date_range = df.select(
    min("order_date").alias("min_date"),
    max("order_date").alias("max_date")
).collect()[0]
print(f"Date range: {date_range['min_date']} to {date_range['max_date']}")

# Check for negative values
negative_amounts = df.filter(col("total_amount") < 0).count()
negative_days = df.filter(col("days_since_last_purchase") < 0).count()
print(f"Negative amounts: {negative_amounts}")
print(f"Negative days since last purchase: {negative_days}")


=== DATA QUALITY CHECKS ===
Customers with multiple rows: 246
Date range: 2023-01-01 to 2024-08-03
Negative amounts: 0
Negative days since last purchase: 0


In [0]:
print("\n=== EDA SUMMARY ===")
print("Key findings for feature engineering:")
print("1. Check class imbalance in target variable")
print("2. Identify outliers in total_amount and days_since_last_purchase") 
print("3. Product category and location show different repeat rates")
print("4. Temporal patterns might be useful features")
print("5. Customer-level aggregations needed for better features")

spark.stop()


=== EDA SUMMARY ===
Key findings for feature engineering:
1. Check class imbalance in target variable
2. Identify outliers in total_amount and days_since_last_purchase
3. Product category and location show different repeat rates
4. Temporal patterns might be useful features
5. Customer-level aggregations needed for better features
